#Setting

In [3]:
!pip install transformers[sentencepiece]
!pip install pytorch-lightning
!pip install konlpy
!pip install pororo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 57.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 104.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.3/800.3 KB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 KB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.4/512.4 KB 19.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 73.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.6/465.6 KB 4

In [4]:
from pororo import Pororo
spacing = Pororo(task="gec", lang="ko")
import numpy as np
import pandas as pd
import torch
#from pytorch_lightning import Trainer
#from pytorch_lightning.callbacks import ModelCheckpoint
#from pytorch_lightning.core.lightning import LightningModule
from torch.utils.data import DataLoader, Dataset
from transformers.optimization import AdamW, get_cosine_schedule_with_warmup
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel
import re
from tqdm import tqdm
from transformers import AutoModel, AutoTokenizer 

As of now, this beta model tries to correct spacing errors in Korean text.


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Train: 인물별 GPT2 모델 생성

In [6]:
#device = torch.device("cpu")
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [7]:
# 허깅페이스 transformers 에 등록된 사전 학습된 koGTP2 토크나이저를 가져온다.
koGPT2_TOKENIZER = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2", bos_token="</s>", eos_token="</s>", unk_token="<unk>", pad_token="<pad>", mask_token='<mask>',)
#토크나이저 저장
koGPT2_TOKENIZER.save_pretrained('tokenizer', legacy_format=False)

Downloading:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/tokenizer.json')

In [8]:
####추가 데이터#####
import urllib.request
urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv",
    filename="addData.csv",
)

add_Data = pd.read_csv("addData.csv")
add_Data.drop('label',axis=1,inplace=True)

In [9]:
Chatbot_Data1 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/(22-2) nlp 학기 프로젝트/데이터/정환_최종.csv", encoding='cp949')
Chatbot_Data2 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/(22-2) nlp 학기 프로젝트/데이터/택_최종.csv", encoding='cp949')
Chatbot_Data3 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/(22-2) nlp 학기 프로젝트/데이터/선우_최종.csv", encoding='cp949')
Chatbot_Data4 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/(22-2) nlp 학기 프로젝트/데이터/동룡_최종.csv", encoding='cp949')

Chatbot_Data1.drop(['role'],axis=1,inplace=True)
Chatbot_Data2.drop(['role'],axis=1,inplace=True)
Chatbot_Data3.drop(['role'],axis=1,inplace=True)
Chatbot_Data4.drop(['role'],axis=1,inplace=True)

Chatbot_Data1=pd.concat([Chatbot_Data1, add_Data], axis = 0)
Chatbot_Data2=pd.concat([Chatbot_Data2, add_Data], axis = 0)
Chatbot_Data3=pd.concat([Chatbot_Data3, add_Data], axis = 0)
Chatbot_Data4=pd.concat([Chatbot_Data4, add_Data], axis = 0)

#Chatbot_Data = Chatbot_Data.sample(n=500)

In [10]:
class ChatbotDataset(Dataset):
    def __init__(self, chats, max_len=40):  # 데이터셋의 전처리를 해주는 부분
        self._data = chats
        self.max_len = max_len
        self.q_token = "<usr>"
        self.a_token = "<sys>"
        self.bos = '</s>'
        self.eos = '</s>'
        self.pad = '<pad>'
        self.mask = '<mask>'
        self.tokenizer = koGPT2_TOKENIZER

    def __len__(self):  # chatbotdata 의 길이를 리턴한다.
        return len(self._data)

    def __getitem__(self, idx):  # 로드한 챗봇 데이터를 차례차례 DataLoader로 넘겨주는 메서드
        turn = self._data.iloc[idx]
        q = turn["Q"]  # 질문을 가져온다.

        a = turn["A"]  # 답변을 가져온다.

        q_toked = self.tokenizer.tokenize(self.q_token + q +self.bos )
        q_len = len(q_toked)

        a_toked = self.tokenizer.tokenize(self.a_token + a + self.eos)
        a_len = len(a_toked)

        #질문의 길이가 최대길이보다 크면
        if q_len > self.max_len:
            a_len = self.max_len - q_len        #답변의 길이를 최대길이 - 질문길이
            if a_len <= 0:       #질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_toked = q_toked[-(int(self.max_len / 2)) :]   #질문길이를 최대길이의 반으로 
                q_len = len(q_toked)
                a_len = self.max_len - q_len              #답변의 길이를 최대길이 - 질문길이
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        #질문의 길이 + 답변의 길이가 최대길이보다 크면
        if q_len + a_len > self.max_len:
            a_len = self.max_len - q_len        #답변의 길이를 최대길이 - 질문길이
            if a_len <= 0:       #질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_toked = q_toked[-(int(self.max_len / 2)) :]   #질문길이를 최대길이의 반으로 
                q_len = len(q_toked)
                a_len = self.max_len - q_len              #답변의 길이를 최대길이 - 질문길이
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        # 답변 labels = [mask, mask, ...., mask, ..., <bos>,..답변.. <eos>, <pad>....]
        labels = [self.mask,] * q_len + a_toked[1:]

        # mask = 질문길이 0 + 답변길이 1 + 나머지 0
        mask = [0] * q_len + [1] * a_len + [0] * (self.max_len - q_len - a_len)
        # 답변 labels을 index 로 만든다.
        labels_ids = self.tokenizer.convert_tokens_to_ids(labels)
        # 최대길이만큼 PADDING
        while len(labels_ids) < self.max_len:
            labels_ids += [self.tokenizer.pad_token_id]

        # 질문 + 답변을 index 로 만든다.    
        token_ids = self.tokenizer.convert_tokens_to_ids(q_toked + a_toked)
        # 최대길이만큼 PADDING
        while len(token_ids) < self.max_len:
            token_ids += [self.tokenizer.pad_token_id]

        #질문+답변, 마스크, 답변
        return (token_ids, np.array(mask), labels_ids)


def collate_batch(batch):
    data = [item[0] for item in batch]
    mask = [item[1] for item in batch]
    label = [item[2] for item in batch]
    return torch.LongTensor(data).to(device), torch.LongTensor(mask).to(device), torch.LongTensor(label).to(device)

In [11]:
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')
train_set1 = ChatbotDataset(Chatbot_Data1, max_len=40)
train_set2 = ChatbotDataset(Chatbot_Data2, max_len=40)
train_set3 = ChatbotDataset(Chatbot_Data3, max_len=40)
train_set4 = ChatbotDataset(Chatbot_Data4, max_len=40)


#윈도우 환경에서 num_workers 는 무조건 0으로 지정, 리눅스에서는 2
train_dataloader1 = DataLoader(train_set1, batch_size=16, num_workers=0, shuffle=True, collate_fn=collate_batch,)
train_dataloader2 = DataLoader(train_set2, batch_size=16, num_workers=0, shuffle=True, collate_fn=collate_batch,)
train_dataloader3 = DataLoader(train_set3, batch_size=16, num_workers=0, shuffle=True, collate_fn=collate_batch,)
train_dataloader4 = DataLoader(train_set4, batch_size=16, num_workers=0, shuffle=True, collate_fn=collate_batch,)

model.to(device)
model.train()

Downloading:   0%|          | 0.00/513M [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [12]:
learning_rate = 3e-5
criterion = torch.nn.CrossEntropyLoss(reduction="none")
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

epoch = 10
Sneg = -1e18

In [13]:
print("start")
for batch_idx, samples in enumerate(train_dataloader1):
    token_ids, mask, label = samples
    print("token_ids ====> ", token_ids)
    print("mask =====> ", mask)
    print("label =====> ", label)
print("end")

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
        [    6,     6,     6,     6,     6,     6,     6,     6,     6,  9265,
          7162,  9208,  8539,  9620, 14821,  8146,  8046, 25856,     1,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3],
        [    6,     6,     6,     6,     6,     6, 16920, 29452,  7801, 25856,
             1,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3],
        [    6,     6,     6,     6,     6,     6,     6,     6, 21737,  9124,
          9658,  8263, 47016, 25856,     1,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3],
        [    

In [ ]:
print ("start")
for epoch in tqdm(range(epoch)):
    for batch_idx, samples in enumerate(train_dataloader1):
        optimizer.zero_grad()
        token_ids, mask, label = samples
        out = model(token_ids)
        out = out.logits      #Returns a new tensor with the logit of the elements of input
        mask_3d = mask.unsqueeze(dim=2).repeat_interleave(repeats=out.shape[2], dim=2)
        mask_out = torch.where(mask_3d == 1, out, Sneg * torch.ones_like(out))
        loss = criterion(mask_out.transpose(2, 1), label)
        # 평균 loss 만들기 avg_loss[0] / avg_loss[1] <- loss 정규화
        avg_loss = loss.sum() / mask.sum()
        avg_loss.backward()
        # 학습 끝
        optimizer.step()
print ("end")

In [ ]:
torch.save(model,'/content/drive/MyDrive/Colab Notebooks/(22-2) nlp 학기 프로젝트/18정환kogpt2.pt')

In [14]:
print("start")
for batch_idx, samples in enumerate(train_dataloader2):
    token_ids, mask, label = samples
    print("token_ids ====> ", token_ids)
    print("mask =====> ", mask)
    print("label =====> ", label)
print("end")

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
         19647,  9341,   389, 10723,   389,  8153,  6889,     1,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3],
        [    2, 26147, 19932,  9098,  7656,  6969,     1,     4, 46435, 21336,
          9122,  8046, 25856,     1,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3],
        [    2, 30652,   376,   376,     1,     4, 30652,   406,   739, 26569,
          8505,   739,   739,   739,  6910,  8338,  7965,  9705,   406,     1,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3],
        [    2, 45893,  9359,  7281, 27436,  8263,   389,     1,     4, 10039,
          788

In [ ]:
print ("start")
for epoch in tqdm(range(epoch)):
    for batch_idx, samples in enumerate(train_dataloader2):
        optimizer.zero_grad()
        token_ids, mask, label = samples
        out = model(token_ids)
        out = out.logits      #Returns a new tensor with the logit of the elements of input
        mask_3d = mask.unsqueeze(dim=2).repeat_interleave(repeats=out.shape[2], dim=2)
        mask_out = torch.where(mask_3d == 1, out, Sneg * torch.ones_like(out))
        loss = criterion(mask_out.transpose(2, 1), label)
        # 평균 loss 만들기 avg_loss[0] / avg_loss[1] <- loss 정규화
        avg_loss = loss.sum() / mask.sum()
        avg_loss.backward()
        # 학습 끝
        optimizer.step()
print ("end")

In [ ]:
torch.save(model,'/content/drive/MyDrive/Colab Notebooks/(22-2) nlp 학기 프로젝트/18택kogpt2.pt')

In [15]:
print("start")
for batch_idx, samples in enumerate(train_dataloader3):
    token_ids, mask, label = samples
    print("token_ids ====> ", token_ids)
    print("mask =====> ", mask)
    print("label =====> ", label)
print("end")

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], device='cuda:0')
label =====>  tensor([[    6,     6,     6,     6,     6,     6,     6,  9564, 11667,  7652,
          7801, 258

In [ ]:
print ("start")
for epoch in tqdm(range(epoch)):
    for batch_idx, samples in enumerate(train_dataloader3):
        optimizer.zero_grad()
        token_ids, mask, label = samples
        out = model(token_ids)
        out = out.logits      #Returns a new tensor with the logit of the elements of input
        mask_3d = mask.unsqueeze(dim=2).repeat_interleave(repeats=out.shape[2], dim=2)
        mask_out = torch.where(mask_3d == 1, out, Sneg * torch.ones_like(out))
        loss = criterion(mask_out.transpose(2, 1), label)
        # 평균 loss 만들기 avg_loss[0] / avg_loss[1] <- loss 정규화
        avg_loss = loss.sum() / mask.sum()
        avg_loss.backward()
        # 학습 끝
        optimizer.step()
print ("end")

In [ ]:
torch.save(model,'/content/drive/MyDrive/Colab Notebooks/(22-2) nlp 학기 프로젝트/18선우kogpt2.pt')

In [16]:
print("start")
for batch_idx, samples in enumerate(train_dataloader4):
    token_ids, mask, label = samples
    print("token_ids ====> ", token_ids)
    print("mask =====> ", mask)
    print("label =====> ", label)
print("end")

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
           406,     1,     3,     3,     3,     3,     3,     3,     3,     3],
        [    2,  9070, 32933,  9122, 25856, 47952,  8806, 50585,   389,  9022,
          6866, 11795,  9182,  7722, 25856,     1,     4,  9108,  9114,  8092,
          9705,   376,   376, 47952, 22633,   739, 11649,  9050,  7810,   406,
          9625, 43049,  9858, 18762, 48906,  8146,   406,     1,     3,     3],
        [    2,   739,  7753,  8210,  7080,   406,     1,     4, 12329, 26382,
          7172,   406,     1,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3]],
       device='cuda:0')
mask =====>  tensor([[0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0

In [ ]:
print ("start")
for epoch in tqdm(range(epoch)):
    for batch_idx, samples in enumerate(train_dataloader4):
        optimizer.zero_grad()
        token_ids, mask, label = samples
        out = model(token_ids)
        out = out.logits      #Returns a new tensor with the logit of the elements of input
        mask_3d = mask.unsqueeze(dim=2).repeat_interleave(repeats=out.shape[2], dim=2)
        mask_out = torch.where(mask_3d == 1, out, Sneg * torch.ones_like(out))
        loss = criterion(mask_out.transpose(2, 1), label)
        # 평균 loss 만들기 avg_loss[0] / avg_loss[1] <- loss 정규화
        avg_loss = loss.sum() / mask.sum()
        avg_loss.backward()
        # 학습 끝
        optimizer.step()
print ("end")

In [ ]:
torch.save(model,'/content/drive/MyDrive/Colab Notebooks/(22-2) nlp 학기 프로젝트/18동룡kogpt2.pt')

#Test

In [17]:
device = torch.device("cpu")
#device = "cuda" if torch.cuda.is_available() else "cpu"
device

tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
                                                    bos_token='</s>', eos_token='</s>', unk_token='<unk>',pad_token='<pad>', mask_token='<mask>')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [ ]:
model1 = torch.load('/content/drive/MyDrive/Colab Notebooks/(22-2) nlp 학기 프로젝트/18정환kogpt2.pt', map_location=device)
model2 = torch.load('/content/drive/MyDrive/Colab Notebooks/(22-2) nlp 학기 프로젝트/18택kogpt2.pt', map_location=device)
model3 = torch.load('/content/drive/MyDrive/Colab Notebooks/(22-2) nlp 학기 프로젝트/18선우kogpt2.pt', map_location=device)
model4 = torch.load('/content/drive/MyDrive/Colab Notebooks/(22-2) nlp 학기 프로젝트/18동룡kogpt2.pt', map_location=device)

#정환 챗봇

In [ ]:
with torch.no_grad():
    while 1:
        q = input("user > ").strip()
        if q == "quit":
            break
        a = ""
        while 1:
            input_ids = torch.LongTensor(tokenizer.encode("<usr>" + q +"<sys>" + a)).unsqueeze(dim=0)
            input_ids.to(device)
            pred = model1(input_ids)
            pred = pred.logits
            gen = tokenizer.convert_ids_to_tokens(torch.argmax(pred, dim=-1).squeeze().numpy().tolist())[-1]
            if gen == '</s>':
                break
            a += gen.replace("▁", " ")
        print("Chatbot > {}".format(spacing(a.strip())))

#택 챗봇

In [ ]:
with torch.no_grad():
    while 1:
        q = input("user > ").strip()
        if q == "quit":
            break
        a = ""
        while 1:
            input_ids = torch.LongTensor(tokenizer.encode("<usr>" + q +"<sys>" + a)).unsqueeze(dim=0)
            input_ids.to(device)
            pred = model2(input_ids)
            pred = pred.logits
            gen = tokenizer.convert_ids_to_tokens(torch.argmax(pred, dim=-1).squeeze().numpy().tolist())[-1]
            if gen == '</s>':
                break
            a += gen.replace("▁", " ")
        print("Chatbot > {}".format(spacing(a.strip())))

#선우 챗봇

In [ ]:
with torch.no_grad():
    while 1:
        q = input("user > ").strip()
        if q == "quit":
            break
        a = ""
        while 1:
            input_ids = torch.LongTensor(tokenizer.encode("<usr>" + q +"<sys>" + a)).unsqueeze(dim=0)
            input_ids.to(device)
            pred = model3(input_ids)
            pred = pred.logits
            gen = tokenizer.convert_ids_to_tokens(torch.argmax(pred, dim=-1).squeeze().numpy().tolist())[-1]
            if gen == '</s>':
                break
            a += gen.replace("▁", " ")
        print("Chatbot > {}".format(spacing(a.strip())))

#동룡 챗봇

In [ ]:
with torch.no_grad():
    while 1:
        q = input("user > ").strip()
        if q == "quit":
            break
        a = ""
        while 1:
            input_ids = torch.LongTensor(tokenizer.encode("<usr>" + q +"<sys>" + a)).unsqueeze(dim=0)
            input_ids.to(device)
            pred = model4(input_ids)
            pred = pred.logits
            gen = tokenizer.convert_ids_to_tokens(torch.argmax(pred, dim=-1).squeeze().numpy().tolist())[-1]
            if gen == '</s>':
                break
            a += gen.replace("▁", " ")
        print("Chatbot > {}".format(spacing(a.strip())))